# Imports

In [1]:
import os
# MASK GPUS
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

import wandb
from wandb.keras import WandbCallback

import config
import utils
import music_model

2023-05-11 09:46:35.724456: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 09:46:36.024806: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-11 09:46:36.058464: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-11 09:46:36.770205: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

# Training function

In [2]:
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATASET_NAME = 'lmd_matched_final_2048_cut'

USE_SMALL_GENRE_SET = DATASET_NAME == 'tf_data7dict'

def get_model_name(use_mask: bool, use_reg: bool, use_mse_for_velocity:bool):
    if use_mask and not use_reg:
        model_name = 'mask_only'
    elif use_reg and not use_mask:
        model_name = 'reg_only'
    elif use_reg and use_mask:
        model_name = 'reg_and_mask'
    else:
        model_name = 'baseline'
    if use_mse_for_velocity:
        model_name += '_with_mse_vel'
    return model_name

def run_training(additional_model_name='', log_name='', model_type='GPT', 
                 use_mask=True, use_reg=True, use_mse_for_velocity=True,
                 use_wandb=True, use_one_gpu=False, seq_len=2048, verbose=True,
                 double_head=False):
    
    if verbose: print("Running setup...")
    model_name = get_model_name(use_mask, use_reg, use_mse_for_velocity)
    
    if verbose: print("\tInstantiating and personalizing Config object...")
    conf = config.Config(config_string="single_instruments_type", 
                         sequence_length=seq_len,
                         root_path=ROOT_PATH, 
                         model_type=model_type,
                         model_name=f'model_{model_type}_{model_name + additional_model_name}')

    # Config object has by default the full list of accepted subgenres and works on multi-gpus
    # If we use the small dataset
    if USE_SMALL_GENRE_SET:
        conf.accepted_subgenres = ['folk', 'nes', 'maestro']

    # If we need to use only the first GPU
    if use_one_gpu:
        conf.GPUS = tf.config.list_physical_devices('GPU')[0]
        conf.BATCH_SIZE = 24
        conf.GLOBAL_BATCH_SIZE = conf.BATCH_SIZE
        conf.num_devices = 1
        print(f"Using only GPU: {conf.GPUS}")
    
    if verbose: print("\tCreating model...")
    if conf.num_devices > 1:
        print("Using multiple GPUs with Mirrored Strategy")
        with conf.training_strategy.scope():
            model = music_model.create_model(conf, 
                                             use_masking_layers=use_mask,
                                             use_regularization=use_reg,
                                             use_mse_for_velocity=use_mse_for_velocity,
                                             double_head=double_head)
    else:
        print("Using single GPU/CPU device")
        model = music_model.create_model(conf, 
                                         use_masking_layers=use_mask,
                                         use_regularization=use_reg,
                                         use_mse_for_velocity=use_mse_for_velocity,
                                         double_head=double_head)
    
    if verbose: print("\tSetupping Wandb logger...")
    if use_wandb:
        wandb_config = {
            'gpus': conf.num_devices,
            'dataset': DATASET_NAME,
            'genres': conf.accepted_subgenres,
            'embedding_size': conf.SINGLE_EMB_SIZE,
            'batch_size': conf.BATCH_SIZE,
            'global_batch_size': conf.GLOBAL_BATCH_SIZE,
            'mse_for_velocity': use_mse_for_velocity,
            'reg_loss_scale': conf.REG_LOSS_SCALE,
            'masking': conf.USE_MASKING,
            'dropout_prob': conf.DROPOUT_VALUE,
            'seq_len': conf.SEQ_LEN,
            'token_dim': conf.TOKEN_DIM,
            'genre_dim': conf.GENRE_DIM,
            'attn_heads': conf.ATTENTION_HEADS,
            'attn_blocks': conf.ATTENTION_BLOCKS,
        }

        if model_type == 'GPT':
            wandb_config['activation_func'] = conf.DECODER_ACTIVATION_FUNCTION
        elif model_type == 'XL':
            wandb_config['sequence_blocks'] = conf.DIV_VAL
            wandb_config['head_dim']  = conf.HEAD_DIM
            wandb_config['inner_dim'] = conf.INNER_DIM
            wandb_config['memory_length'] = conf.MEMORY_LEN

        run = wandb.init(project="Music Generation", entity="marcello-e-federico",
                         group=model_name, job_type='train', config=wandb_config,
                         name=log_name if log_name != '' else None)
    
    if verbose: print("\tObtaining dataset...")
    dataset_path = conf.dataset_paths[DATASET_NAME]
    train_dataset, val_dataset, test_dataset = utils.get_dataset_splits(dataset_path, conf)
    
    if verbose: print("\tSetupping callbacks...")
    callbacks = conf.MODEL_CALLBACKS
    if use_wandb:
        callbacks.append(WandbCallback(
            save_model=False, save_graph=False,
            log_weights=False
        ))
    
    if verbose: print("Training start")
    history = model.fit(
        train_dataset,
        epochs = 100,
        callbacks = callbacks,
        validation_data = val_dataset,
        # initial_epoch = initial_epoch # change if resuming from previous checkpoint
    )
    if verbose: print("Training finished")
    
    if use_wandb:
        if verbose: print("\tClosing Wandb logger...")
        run.finish()
        
    if verbose: print("\tClearing session...")
    K.clear_session()
        
    return model, history

# Trainings

In [ ]:
# print("TRAINING BASELINE MODEL\n\n")
# model_baseline_mse_vel, history_baseline_mse_vel = run_training(additional_model_name='lmd_matched_2048', log_name='', model_type='GPT', 
#                                                         use_mask=False, use_reg=False, use_mse_for_velocity=True, use_wandb=True, 
#                                                         use_one_gpu=True, seq_len=2048, verbose=True)

# print("=================================\n")
# print("TRAINING REG ONLY MODEL\n\n")    
# model_reg_mse_vel, history_reg_mse_vel = run_training(additional_model_name='lmd_matched_2048', log_name='', model_type='GPT', 
#                                                         use_mask=False, use_reg=True, use_mse_for_velocity=True, use_wandb=True, 
#                                                         use_one_gpu=True, seq_len=2048, verbose=True)

print("=================================\n")
print("TRAINING MASK ONLY MODEL\n\n")
model_mask_mse_vel, history_mask_mse_vel = run_training(additional_model_name='lmd_matched_2048', log_name='', model_type='GPT', 
                                                        use_mask=True, use_reg=False, use_mse_for_velocity=True, use_wandb=True, 
                                                        use_one_gpu=True, seq_len=2048, verbose=True, double_head=False)

# print("=================================\n")
# print("TRAINING MASK AND REG MODEL\n\n")
# model_mask_reg_mse_vel, history_mask_reg_mse_vel = run_training(additional_model_name='lmd_matched_2048', log_name='', model_type='GPT', 
#                                                         use_mask=True, use_reg=True, use_mse_for_velocity=True, use_wandb=True, 
#                                                         use_one_gpu=True, seq_len=2048, verbose=True)


TRAINING MASK ONLY MODEL


Running setup...
	Instantiating and personalizing Config object...
Using only GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
	Creating model...
Using single GPU/CPU device


2023-05-11 09:47:06.046164: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 09:47:06.721337: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30970 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


	Setupping Wandb logger...


wandb: Currently logged in as: volpepe (marcello-e-federico). Use `wandb login --relogin` to force relogin


	Obtaining dataset...
	Setupping callbacks...
Training start
Epoch 1/100
 839/2317 [=========>....................] - ETA: 1:12:02 - loss: 0.9148 - type_loss: 0.0022 - measure_loss: 0.1422 - beat_loss: 0.0371 - position_loss: 0.1142 - duration_loss: 0.1667 - pitch_loss: 0.1633 - instrument_loss: 0.1358 - velocity_loss: 1.0083e-06 - key_sign_loss: 0.0333 - time_sign_loss: 0.0158 - tempo_loss: 0.1042

In [ ]:
# print("TRAINING BASELINE MODEL\n\n")
# model_baseline_mse_vel, history_baseline_mse_vel = run_training(additional_model_name='_lmd_matched_2048_double_head', log_name='', model_type='GPT', 
#                                                         use_mask=False, use_reg=False, use_mse_for_velocity=True, use_wandb=True, 
#                                                         use_one_gpu=True, seq_len=2048, verbose=True, double_head=True)

# print("=================================\n")
# print("TRAINING REG ONLY MODEL\n\n")    
# model_reg_mse_vel, history_reg_mse_vel = run_training(additional_model_name='_lmd_matched_2048_double_head', log_name='', model_type='GPT', 
#                                                         use_mask=False, use_reg=True, use_mse_for_velocity=True, use_wandb=True, 
#                                                         use_one_gpu=True, seq_len=2048, verbose=True, double_head=True)